安装opencompass：Kaggle上已经为我们准备好了其他常用包，只需安装opencompass用于评测即可。如果不在Kaggle上运行，则还需要安装其他必要包。

In [1]:


import argparse
from dataclasses import dataclass, field
from typing import Optional, List, Dict
import sys
import torch
from transformers import TrainingArguments, HfArgumentParser, Trainer, AutoTokenizer, AutoModelForCausalLM
import datasets
import os
# set visible gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "4"


/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import textwrap

# 在需要打印的地方
def pretty_print(text, width=80):
    print("\n".join(textwrap.wrap(text, width=width)))


# 指令微调

In [3]:
# Define the arguments required for the main program.
# NOTE: You can customize any arguments you need to pass in.
@dataclass
class ModelArguments:
    """Arguments for model
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the LLM to fine-tune or its name on the Hugging Face Hub."
        }
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype."
            ),
            "choices": ["bfloat16", "float16", "float32"],
        },
    )
    # TODO: add your model arguments here
    pass


@dataclass
class DataArguments:
    """Arguments for data
    """
    dataset_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The path to the fine-tuning dataset or its name on the Hugging Face Hub."
        }
    )
    # TODO: add your data arguments here
    max_length: int = field(
        default=512,
        metadata={
            "help": "The max length of tokenized data."
        }
    )
    skip_too_long: Optional[bool] = field(
        default = False,
        metadata = {
            "help" : "whether to skip those longer than max length of tokenized data"
        }
    )

In [14]:
def load_single_model(model_path,model_name,torch_dtype,trust_remote_code,device_map,use_cache):
    return AutoModelForCausalLM.from_pretrained(
            pretrained_model_name_or_path=model_path, 
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,  # Qwen模型需要这个参数
            device_map=device_map,  # 可选，用于自动处理模型加载到设备
            use_cache=use_cache
        )
def loading(model_list):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    parser = HfArgumentParser(dataclass_types=[ModelArguments, DataArguments, TrainingArguments])
    model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    
    dataset = datasets.load_dataset(path='csv', data_files=data_args.dataset_path)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_args.model_name_or_path)
    for model_dict in model_list:
        if model_dict['enable'] == True:
            model_name = model_dict['name']
            model_path = model_dict['path']
            model = load_single_model(model_path,model_name,model_args.torch_dtype,True,"auto",False)
            model = model.to(device)
            # model_list[model_name] = model
            model_dict['model'] = model
    return dataset,model_list,tokenizer
        
    # model_sft = AutoModelForCausalLM.from_pretrained(
    #         pretrained_model_name_or_path=sft_model_path, 
    #         torch_dtype=model_args.torch_dtype,
    #         trust_remote_code=True,  # Qwen模型需要这个参数
    #         device_map="auto",  # 可选，用于自动处理模型加载到设备
    #         use_cache=False
    #     )
    # model_sft_2 = AutoModelForCausalLM.from_pretrained(
    #         pretrained_model_name_or_path=sft_model_path_2, 
    #         torch_dtype=model_args.torch_dtype,
    #         trust_remote_code=True,  # Qwen模型需要这个参数
    #         device_map="auto",  # 可选，用于自动处理模型加载到设备
    #         use_cache=False
    #     )       
    # return dataset,model_plm.to(device),model_sft.to(device),model_sft_2.to(device),tokenizer

def test_single_model(model,inputs,tokenizer,device):
    
    inputs['input_ids'] = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    max_tokens = 256
    pad_token_id = tokenizer.pad_token_id
    generate_ids = model.generate(inputs['input_ids'], attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=max_tokens)
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs['input_ids'], generate_ids)]
    generated_text_plm = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return generated_text_plm

def test(dataset,model_list,tokenizer,your_input=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if type(your_input)==int:
        sample = dataset['train'][your_input]
        output_text = sample["output"]
        
        text = "instruction: " + sample["instruction"] if sample["instruction"] else ""
        text += "\n input: " + sample["input"] if sample["input"] else ""
        
        print("Text: ", text)
        print("GT: ", output_text)
        print("===")
        # 不进行 padding，只进行截断
        inputs = tokenizer(text, return_tensors="pt")
    elif type(your_input)==str:
        inputs=tokenizer(your_input, return_tensors="pt")
    
    # # 查看生成的输入 IDs
    # print("Input IDs:", inputs['input_ids'])
    
    # # 查看生成的 attention_mask，不会被padding
    # print("Attention Mask:", inputs['attention_mask'])
    
    # # 查看 pad_token_id
    # print("Pad Token ID:", tokenizer.pad_token_id)

    for model_dict in model_list:
        if model_dict['enable'] == True:
            model_name = model_dict['name']
            model = model_dict['model']
            generated_text = test_single_model(model,inputs,tokenizer,device)
            print(f"model:{model_name} done")
            pretty_print(f"generated_text:{generated_text}")
            print(".....")
    
    # generate_ids = model_plm.generate(inputs['input_ids'], attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=max_tokens)
    # generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs['input_ids'], generate_ids)]
    # generated_text_plm = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    
    # pretty_print(f"plm:{generated_text_plm}\n")
    # print(".....")
    # generate_ids = model_sft.generate(inputs['input_ids'], attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=max_tokens)
    # generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs['input_ids'], generate_ids)]
    # generated_text_sft = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    
    # pretty_print(f"sft:{generated_text_sft}\n")
    # print(".....")

    # if model_sft_2 is not None:
    #     generate_ids = model_sft_2.generate(inputs['input_ids'], attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=max_tokens)
    #     generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs['input_ids'], generate_ids)]
    #     generated_text_sft_2 = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    #     pretty_print(f"sft_2:{generated_text_sft_2}\n")
    #     print("==========")
    # return generated_text_plm,generated_text_sft,generated_text_sft_2


# new api:

In [16]:
model_list = [
    {
        'name':'sft_p',
        'path':'/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/1430/checkpoint-50000',
        'enable':True
    },
    {
        'name':'sft_a',
        'path':'/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/0.5B/checkpoint-50000',
        'enable':True
    },
    {
        'name':'peft',
        'path':'/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/peft_output/checkpoint-30000',
        'enable':True
    }
]
sys.argv = [
    "notebook", 
    # "--arg1", "value1",
    # "--arg2", "value2",
    # ...
    ### cjy
    "--model_name_or_path", "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/1430/checkpoint-50000",
    "--dataset_path", "/home/xiaxinyuan/.cache/kagglehub/datasets/thedevastator/alpaca-language-instruction-training/versions/2/train.csv",
    "--torch_dtype", "bfloat16", #see Qwen2.5-0.5B/config.json?
    "--output_dir", "output/1227/", # --output_dir 参数在 TrainingArguments 中有
    "--remove_unused_columns", "False", #ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [output, instruction, input]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`
    "--max_length", "512",
    ### xxy
    "--skip_too_long","True",
    "--learning_rate","1e-5",
    "--lr_scheduler_type","cosine",
    "--optim", "adamw_hf",
    "--warmup_ratio", "0.03",
    "--weight_decay", "0.003",
    ### xxy
    ### cjy
    "--per_device_train_batch_size", "4",  # 设置训练的 batch size
    "--per_device_eval_batch_size", "4", 
    "--save_steps", "1000",
    "--save_total_limit", "3",
    "--num_train_epochs", "3",  # 通常3-5个epoch即可收敛，长时间训练可能会过拟合 
    "--bf16","True",  # 开启混合精度加速
]
dataset,model_list,tokenizer = loading(model_list)

instruction = ""
input_text = None
print(f"instruction: {instruction}\ninput: {input_text}")
test(dataset,model_list,tokenizer,your_input=instruction)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.21it/s]


instruction: 请祝福我的列表跨年快乐
input: None
model:sft_p done
generated_text:，辞旧迎 new year, happy!  2021, is a brand new year, let's all be
happy and lucky, and have a great new year!  2020, is a year of new beginnings,
let's all be ready for a new start, and have a great year!  2019, is a year of
growth and change, let's all be inspired and motivated, and have a great year
ahead!  2018, is a year of learning and growth, let's all be curious and open to
new experiences, and have a great year ahead!  2017, is a year of joy and
laughter, let's all be filled with happiness and joy, and have a great year
ahead!  2016, is a year of reflection and self-care, let's all be mindful and
present in the moment, and have a great year ahead!  2015, is a year of
opportunity and adventure, let's all be open to new challenges, and have a great
year ahead!  2014, is a year of growth and learning, let's all be inspired and
motivated, and have a great year ahead
.....
model:sft_a done
generated_text: input: None


# Past API:

In [ ]:

# Pass your training arguments.
# NOTE [IMPORTANT!!!] DO NOT FORGET TO PASS PROPER ARGUMENTS TO SAVE YOUR CHECKPOINTS!!!
sys.argv = [
    "notebook", 
    # "--arg1", "value1",
    # "--arg2", "value2",
    # ...
    ### cjy
    "--model_name_or_path", "/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/1430/checkpoint-50000",
    "--dataset_path", "/home/xiaxinyuan/.cache/kagglehub/datasets/thedevastator/alpaca-language-instruction-training/versions/2/train.csv",
    "--torch_dtype", "bfloat16", #see Qwen2.5-0.5B/config.json?
    "--output_dir", "output/1227/", # --output_dir 参数在 TrainingArguments 中有
    "--remove_unused_columns", "False", #ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [output, instruction, input]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`
    "--max_length", "512",
    ### xxy
    "--skip_too_long","True",
    "--learning_rate","1e-5",
    "--lr_scheduler_type","cosine",
    "--optim", "adamw_hf",
    "--warmup_ratio", "0.03",
    "--weight_decay", "0.003",
    ### xxy
    ### cjy
    "--per_device_train_batch_size", "4",  # 设置训练的 batch size
    "--per_device_eval_batch_size", "4", 
    "--save_steps", "1000",
    "--save_total_limit", "3",
    "--num_train_epochs", "3",  # 通常3-5个epoch即可收敛，长时间训练可能会过拟合 
    "--bf16","True",  # 开启混合精度加速
]
dataset,model_plm,model_sft,model_sft_2,tokenizer=loading(plm_model_path="/home/xiaxinyuan/.cache/kagglehub/models/qwen-lm/qwen2.5/transformers/0.5b/1/", # 修改为你的模型路径
                                    sft_model_path="/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/1430/checkpoint-50000",
                                    sft_model_path_2="/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/output/0.5B/checkpoint-50000") # 修改为你的模型路径

In [49]:
instruction_set = [
    "college econometrics\n \
    Which of the following statements concerning the regression population and sample is FALSE? \
    \nA. The population is the total collection of all items of interest \
    \nB. The population can be infinite \
    \nC. In theory, the sample could be larger than the population \
    \nD. A random sample is one where each individual item from the population is equally likely to be drawn. \
    \nC. In theory, the sample could be larger than the population",
]

In [54]:
# instruction_set = [
#     "Which of the following statements concerning the regression population and sample is FALSE?\n A. The population is the total collection of all items of interest\n B. The population can be infinite\n C. In theory, the sample could be larger than the population\n D. A random sample is one where each individual item from the population is equally likely to be drawn."
# ]
import pandas as pd
def get_template(instruction,choice,field,ans,mode = 'infer'):
    template_set = [
        # f"HUMAN: There is a single choice question about {field}. \nQ:{instruction}: {choice} ",
        # f"HUMAN: There is a single choice question about {field}. \nQ:{instruction}: {choice}\nBOT: ",
        # f"HUMAN: There is a single choice question about college physics. Q:{instruction}: {choice}\nBOT: ",
        # f"HUMAN: There is a single choice question about college physics. \nQ:{instruction}: {choice}\nBOT: ",
        # f"HUMAN: There is a single choice question about college mathematics. Q:{instruction}: {choice}\nBOT: ",
        # f"HUMAN: There is a single choice question about {field}.\nQ:{instruction}: {choice}\nLet's think step by step. A:",
        # f"HUMAN: There is a single choice question about {field}.\nQ:{instruction}: {choice}\nLet's think step by step. BOT: A:",
        # f"HUMAN: There is a single choice question about {field}.\nQ: {instruction}:\n{choice}\nLet's think step by step. BOT:",
        # f"HUMAN: There is a single choice question about {field}.\nQ: {instruction}:\n{choice}\nA:",
        f"HUMAN: There is a single choice question about {field}.\nQuestion: {instruction}. \n{choice}\n Why the answer is {ans}? \n Think step by step. BOT: ",
    ]
    return template_set
for instruction in instruction_set: 
    ins = instruction.split("\n")
    area = ins[0]
    instruction = ins[1]
    choice = ins[2:6]
    ans=ins[6]
    # print(area)
    formatted_text = '\n'.join(choice.strip() for choice in choice)
    final_instructions = get_template(instruction,formatted_text,area,ans)
    for final_instruction in final_instructions:
        # pretty_print(final_instruction)
        print(final_instruction)
        plm_ans, sft1_ans, sft2_ans = test(dataset=dataset,
            model_plm=model_plm,
            model_sft=model_sft,
            model_sft_2=model_sft_2,
            tokenizer=tokenizer,
            your_input=final_instruction) # 如果your_input是数字，则是被理解dataset中的index，即问alpaca中的第your_input个问题；如果是字符串，则是输入的文本
        # save template set and answer in a row in excel
        # df = pd.DataFrame({'template': [final_instruction], 'plm_ans': [plm_ans], 'sft1_ans': [sft1_ans], 'sft2_ans': [sft2_ans]})
        # df.to_excel('case_study.xlsx', index=False)


HUMAN: There is a single choice question about college econometrics.
Question:      Which of the following statements concerning the regression population and sample is FALSE?     . 
A. The population is the total collection of all items of interest
B. The population can be infinite
C. In theory, the sample could be larger than the population
D. A random sample is one where each individual item from the population is equally likely to be drawn.
 Why the answer is C. In theory, the sample could be larger than the population? 
 Think step by step. BOT: 
plm:1. The population is the total collection of all items of interest. 2. In
theory, the sample could be larger than the population. 3. A random sample is
one where each individual item from the population is equally likely to be
drawn. 4. The population can be infinite. 5. The population is the total
collection of all items of interest. 6. The population can be infinite. 7. In
theory, the sample could be larger than the population. 8. A

In [8]:
for instruction in instruction_set: 
    ins = instruction.split("\n")
    choices = ins[1:]
    formatted_text = ' '.join(choice.strip() for choice in choices)
    print(formatted_text)

The primary source of the Sun’s energy is a series of thermonuclear reactions in which the energy produced is c^2 times the mass difference between A. two hydrogen atoms and one helium atom B. four hydrogen atoms and one helium atom C. six hydrogen atoms and two helium atoms D. three helium atoms and one carbon atom 


In [9]:

instruction = "Write a humerous joke"
input_text = None
print(f"instruction: {instruction}\ninput: {input_text}")


instruction: Write a humerous joke
input: None


In [ ]:

instruction = "Write a humerous joke"
model_list = [
    {
        'name':'peft',
        'model':model_plm
    },
    {
        'name':'sft',
        'model':model_sft
    }
]
dataset,model_list,tokenizer = loading(model_list,data_args)


In [1]:
from opencompass.models import HuggingFaceCausalLM
from opencompass.datasets import (
    TruthfulQADataset,
    CivilCommentsDataset,
    CrowspairsDatasetV2,
    CrowspairsDatasetCN,
    CValuesDataset,
    JigsawMultilingualDataset,  # 有毒内容生成评估
)
from opencompass.runners import LocalRunner
from opencompass.summarizers import ExampleSummarizer

def evaluate_safety():
    # 模型配置保持不变
    model_cfg = dict(
        type=HuggingFaceCausalLM,
        path="output/0.5B/checkpoint-50000",
        tokenizer_kwargs={
            'padding_side': 'left',
            'truncation': 'left'
        },
        max_seq_len=2048,
        batch_size=4,
        model_kwargs={'device_map': 'auto'},
    )

    # 修改为安全相关的数据集配置
    dataset_cfgs = [
        dict(
            type=TruthfulQA,
            path='truthfulqa',
            name='truthfulqa',
            ppl_eval=True,
            # 评估模型回答的真实性
        ),
        dict(
            type=CivilComments,
            path='civil_comments',
            name='civil_comments',
            ppl_eval=True,
            # 评估模型对有毒内容的处理
        ),
        dict(
            type=HelmHarm,
            path='helm/harm',
            name='helm_harm',
            ppl_eval=True,
            # 评估模型的安全性
        ),
        dict(
            type=BBQPPl,
            path='bbq',
            name='bbq',
            ppl_eval=True,
            # 评估模型的偏见
        ),
        dict(
            type=ToxiGen,
            path='toxigen',
            name='toxigen',
            ppl_eval=True,
            # 评估有毒内容生成
        ),
    ]

    # 修改评测输出路径
    eval_cfg = dict(
        work_dir="/ssd/xiaxinyuan/code/CS3602_NLP_Final_Project/evals/safety",
        summarizer=dict(type=ExampleSummarizer),
        debug=True
    )

    # 创建评测实例
    from opencompass import Evaluator
    evaluator = Evaluator(
        model_cfg=model_cfg,
        dataset_cfgs=dataset_cfgs,
        eval_cfg=eval_cfg
    )

    # 运行评测
    results = evaluator.run()
    return results
# 运行评测
results = evaluate_safety()

# 查看具体数据集的结果
print("TruthfulQA结果:", results['truthfulqa'])
print("有毒内容处理结果:", results['civil_comments'])
print("安全性评估结果:", results['helm_harm'])

/home/xiaxinyuan/.local/lib/python3.10/site-packages/opencompass/__init__.py:19: UserWarning: Starting from v0.4.0, all AMOTIC configuration files currently located in `./configs/datasets`, `./configs/models`, and `./configs/summarizers` will be migrated to the `opencompass/configs/` package. Please update your configuration file paths accordingly.
  _warn_about_config_migration()
/home/xiaxinyuan/.conda/envs/dino/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'TruthfulQA' from 'opencompass.datasets' (/home/xiaxinyuan/.local/lib/python3.10/site-packages/opencompass/datasets/__init__.py)